<img src="http://curiousily.com/assets/11.tensorflow_for_hackers_part_1_files/tensors_flowing.gif" width="50%" height="40%"/>

# Graphs

Everything you do in TensorFlow is based on computational graphs. Each node of the graph defines an operation. The operation can be any function - addition, multiplication or some funky custom function you wrote.

## Tensors

You can give a scalar value, vector, matrix or a Tensor to the operations in the graph. Tensors are just multi-dimensional matrices.

Let's have a look at an example computational graph:

<img src="https://cdn-images-1.medium.com/max/1600/1*mvhm5_r6LY-eHsin21RJTg.png" height="80%" width="80%"/>

How does this look like in TensorFlow?

In [ ]:
import tensorflow as tf

t1 = tf.Variable(2)
t2 = tf.Variable(3)

res = t1 * t2

That was easy, right? No graphs, nothing seriously strange. And now for the result:

In [ ]:
res

That must be how new math is developed. $2 \times 3 = \text{tf.Tensor}$?

First, let's see the hidden `default` graph.

In [ ]:
t1.graph is tf.get_default_graph()

In [ ]:
res.graph is tf.get_default_graph()

All your Tensors are attached to the mystical `default` graph. What about the result?

# Sessions

See, the code above didn't perform any computation. You just created a computational graph. If you want to do something meaningful you need a `session`. Sessions are used for graph evaluation, think variable initialization and actually computing `res`.

A TensorFlow session takes care of running operations (on GPUs and CPUs) and holding variable values. Let's see how this is done:

In [ ]:
with tf.Session() as sess:
    t1.initializer.run()
    t2.initializer.run()
    print(f"Multiplication result: {res.eval()}")

We initialized all our tensors and evaluated the result. Here is a somewhat shorter version that initializes all global variables automagically:

In [ ]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run() # or sess.run(init)
    print(f"Multiplication result: {res.eval()}")

# Variables

Variables are just... eh, variables? By definition, you can change their values. That is quite useful, since tensors and operators are immutable. Let's create a normally distributed $3\times4$ matrix with $mean=0$ and $std=100$

In [ ]:
normal = tf.Variable(tf.random_normal([3, 4], mean=0.0, stddev=100.0))

In [ ]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print(normal.eval())

Now let's change it:

In [ ]:
normal = normal.assign(tf.Variable(tf.random_normal([3, 4], mean=0.0, stddev=100.0)))

In [ ]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print(normal.eval())

Hard, right?

# Simple Linear Regression

$$Y = aX + b$$

Where Y is the dependent and X is the independent variable. Our task is to adjust the parameters a - “slope” and b - “intercept” so that we best describe the data using a line.

For our example, let’s find out how eating burgers affect your resting heart rate. The data will be simulated, so no conclusions, please! Our data represents the average number of burgers eaten per day.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

X = np.random.rand(100).astype(np.float32)

The slope and intercept we are looking for are respectively $a=50$ and $b=40$

In [ ]:
a = 50
b = 40
Y = a * X + b

Let’s have a look at what our model should figure out:

In [ ]:
plt.plot(X, Y);

Let’s make things a tiny bit more interesting by adding a bit of noise to our dependent variable.

In [ ]:
Y = np.vectorize(lambda y: y + np.random.normal(loc=0.0, scale=0.05))(Y)

## Using TensorFlow for Simple Linear Regression

In [ ]:
tf.reset_default_graph()

a_var = tf.Variable(1.0)
b_var = tf.Variable(1.0)
y_var = a_var * X + b_var

Our task will be to minimize the mean squared error or in TensorFlow parlance - reduce the mean.

In [ ]:
loss = tf.reduce_mean(tf.square(y_var - Y))

So, let’s try to minimize it using gradient descent.

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(loss)

Let’s use our optimizer for 300 steps of learning

In [ ]:
TRAINING_STEPS = 300
results = []
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(TRAINING_STEPS):
        results.append(sess.run([train, a_var, b_var])[1:])

Let’s get the final and best predictions for $a$ and $b$

In [ ]:
final_pred = results[-1]
a_hat = final_pred[0]
b_hat = final_pred[1]
y_hat = a_hat * X + b_hat

print(f"a_hat: {a_hat} b_hat: {b_hat}")

Let's see how the predicted model fits the actual data:

In [ ]:
plt.plot(X, Y);
plt.plot(X, y_hat);

That’s a nice fit. Those two lines overlap pretty good, what did you expect? Pretty good for a couple of lines of code.